# Segmenting and Clustering Neighborhoods in Toronto

In [15]:
# Do imports 
from bs4 import BeautifulSoup
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 


## Get web data from Wiki page

In [16]:
# capture html  from Toronto webpage
import requests
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url)
html_doc = r.text


In [17]:
#parse the html
soup = BeautifulSoup(html_doc, 'html.parser')

## Extract the postal code data and store in a dataframe

In [18]:
# Extract the postal code table
table_contents=[]
table=soup.find('table')
# create a dictionary called cell having 3 keys PostalCode, Borough and Neighborhood.
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)


In [19]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.shape

(103, 3)

## Add the Lat/Lon data

In [23]:
# get lat/Lon data
!wget -q -O 'torontoLatLon.csv' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv
torontoLatLon_df = pd.read_csv('torontoLatLon.csv')
torontoLatLon_df.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)
print("Shape of Lat/Lon Data:", torontoLatLon_df.shape)
torontoLatLon_df.head()

Shape of Lat/Lon Data: (103, 3)


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [24]:
# Merge the Lat/Lon data with Borough/Neighborhood
df_all = pd.merge(df, torontoLatLon_df, on="PostalCode")
print("Shape of merged data:", df_all.shape)
df_all.head()

Shape of merged data: (103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
